In [1]:
import sys
import importlib
importlib.reload(sys)
sys.path.insert(0,'../')
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics 
import os
currentpath = os.getcwd()
parentdir = os.path.dirname(currentpath)

../scripts/functions.py:80: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [2]:
importlib.reload(f)

../scripts/functions.py:96: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  self.diag_mels = diag_mels


<module 'scripts.functions' from '../scripts/functions.py'>

In [2]:
from conf import *
from numba import njit, jit

In [59]:
P = np.load(parentdir+'/save/dynamics/h=1.0V=0.9l=[12, 12]/P_n=1.0e+04_0.npy')
T = np.load(parentdir+'/save/dynamics/h=1.0V=0.9l=[12, 12]/T_n=1.0e+04_0.npy')

In [61]:
import multiprocessing as mp
N = samples_state.shape[0]
queue = []
process = []
n = 12
index = np.round(np.linspace(0, N, n + 1)).astype(np.int)

NameError: name 'samples_state' is not defined

In [62]:
@njit
def cal_mels(P, T, t_list):
    mels_list = np.zeros((t_list.shape[0],)+P.shape[1:] , dtype=P.dtype)
#     num_samples = np.zeros_like(T, dtype=nb.boolean)
#     sections = np.zeros(t_list.shape[0])
    t_list_ = t_list.reshape(-1,1)

    S = 0
    for n in range(T.shape[0]-1):
        
#         A = T <= t_list[n]
#         num_samples = np.logical_or(A, )
#         ind1, ind2 = np.where(np.logical_xor(A[1:,:],A[:-1,:]))
#     #     sections[n] = S
#         n_state = ind1.shape[0]

        index = np.logical_and(T[n] <= t_list_,  t_list_ < T[n+1])

        for i in range(index.shape[1]):
            mels_list[index[:,i],i] = P[n,i]
    return mels_list

In [63]:
length = [12, 12]
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
hex_ = nk.machine.new_hex(np.array(length))


l = []
for i in range(length[0]):
    for j in range(length[1]):
        l.append([i, j])
l = np.array(l)
a = 0
a_ = [a for _ in range(np.prod(length))]
a = np.array(a_)



edges, colors = hex_.dimer_corr(l,a)
operators = f.return_spin_corr(hi, edges, colors)

t_list = np.linspace(0, 1, 50)

P_list= cal_mels(P, T, t_list)
P_list_ = P_list.reshape(-1,P_list.shape[-1])
num_samples = (P_list[:,:,0]!=0).sum(axis=1)



In [64]:
P_list.shape

(50, 9984, 288)

In [65]:
sections1 = np.arange(P.shape[1])
sections2 = np.zeros(P_list_.shape[0])

_, mels1 = operators[0].get_conn_flattened(P[0,:,:], sections1)
sub1 = mels1.mean().real
mels1 = mels1.real


dimer_corr = np.zeros((length[0],length[1],t_list.shape[0]))
dimer_std = np.zeros((length[0],length[1],t_list.shape[0]))

# for l1 in range(length[0]):
#     for l2 in range(length[1]):
l1 = 0
l2 = 9
print(l1,l2)

sub2 = operators[l1 * length[1] + l2].get_conn_flattened(P[0,:,:], sections1)[1].mean().real

_, mels2_ = operators[l1 * length[1] + l2].get_conn_flattened(P_list_, sections2)
mels2 = mels2_.reshape(P_list.shape[0], P_list.shape[1]).real
dimer_corr[l1,l2] = (mels2 * mels1).sum(axis=1)/num_samples - sub1*sub2
dimer_std[l1,l2] = (mels2 * mels1).std(axis=1) 

0 9


In [66]:
sub1*sub2

0.02751068376068376

In [67]:
(mels2 * mels1).sum(axis=1)/num_samples

array([0.02654247, 0.02734375, 0.02804487, 0.02814503, 0.02834535,
       0.02924679, 0.02984776, 0.0302484 , 0.03084936, 0.03125313,
       0.03156313, 0.0323842 , 0.032999  , 0.03381497, 0.03443775,
       0.03535556, 0.0360804 , 0.03662709, 0.03686914, 0.03762356,
       0.03877613, 0.03965203, 0.03964311, 0.04014228, 0.04126337,
       0.04178159, 0.04260549, 0.04316547, 0.04343789, 0.04332055,
       0.04329725, 0.04330091, 0.04340497, 0.04332627, 0.04337632,
       0.04408452, 0.04398098, 0.0445943 , 0.04461741, 0.04528885,
       0.04521098, 0.04521935, 0.04616083, 0.04699291, 0.04763552,
       0.04774103, 0.04734492, 0.04724036, 0.04724787, 0.04787879])

In [29]:
sub1*sub2

0.027656831057897763

In [68]:
num_samples

array([9984, 9984, 9984, 9984, 9984, 9984, 9984, 9984, 9984, 9983, 9980,
       9974, 9970, 9966, 9960, 9956, 9950, 9938, 9927, 9914, 9903, 9886,
       9863, 9840, 9815, 9789, 9764, 9730, 9692, 9649, 9608, 9561, 9492,
       9440, 9383, 9323, 9254, 9194, 9122, 9053, 8958, 8890, 8817, 8746,
       8670, 8588, 8512, 8425, 8339, 8250])

In [11]:
corr[0][0, 4]

array([-1.00325786e-03, -1.10341811e-03, -9.03097600e-04, -1.20357837e-03,
       -1.20357837e-03, -1.20357837e-03, -8.02937344e-04, -9.03097600e-04,
       -9.03097600e-04, -1.30373863e-03, -1.20357837e-03, -1.10078945e-03,
       -1.19834058e-03, -1.19572089e-03, -1.29330108e-03, -1.08763821e-03,
       -1.18261460e-03, -9.79490506e-04, -9.79490506e-04, -1.27501524e-03,
       -1.16686969e-03, -8.65966978e-04, -9.55680227e-04, -1.24884803e-03,
       -1.04018441e-03, -9.37131370e-04, -8.31395049e-04, -7.20267143e-04,
       -5.08537266e-04, -6.80080278e-04, -7.64656199e-04, -8.41323866e-04,
       -1.02430808e-03, -1.30584130e-03, -1.08277903e-03, -1.56178547e-03,
       -1.54352327e-03, -1.71736639e-03, -1.17642264e-03, -1.56122095e-03,
       -1.63663586e-03, -1.70177041e-03, -1.56327954e-03, -1.73054663e-03,
       -1.59170064e-03, -1.85139485e-03, -1.48864354e-03, -1.12432560e-03,
       -1.06758171e-03, -7.87587787e-04, -7.46357477e-04, -8.95866863e-04,
       -1.34706919e-03, -

In [12]:
corr[3]

array([9984, 9984, 9984, 9984, 9984, 9984, 9984, 9984, 9984, 9984, 9984,
       9983, 9982, 9981, 9980, 9978, 9976, 9975, 9975, 9973, 9970, 9970,
       9966, 9963, 9960, 9959, 9957, 9953, 9949, 9938, 9932, 9923, 9916,
       9908, 9900, 9890, 9883, 9872, 9859, 9851, 9841, 9827, 9813, 9799,
       9785, 9767, 9746, 9725, 9704, 9677, 9662, 9641, 9616, 9586, 9559,
       9534, 9508, 9481, 9451, 9428, 9400, 9367, 9329, 9293, 9267, 9243,
       9208, 9182, 9142, 9107, 9072, 9039, 9004, 8966, 8936, 8898, 8860,
       8820, 8790, 8737, 8705, 8668, 8635, 8604, 8553, 8507, 8468, 8422,
       8386, 8341, 8297, 8250, 8207, 8157, 8106, 8065, 8023, 7969, 7922,
       7875, 7830, 7787, 7742, 7693, 7656, 7609, 7557, 7503, 7452, 7404,
       7360, 7296, 7253, 7208, 7175, 7123, 7081, 7039, 6994, 6943, 6900,
       6853, 6792, 6747, 6711, 6674, 6628, 6590, 6543, 6497, 6456, 6408,
       6357, 6323, 6284, 6239, 6193, 6157, 6102, 6052, 6013, 5972, 5929,
       5872, 5825, 5786, 5742, 5703, 5660, 5610, 55

In [16]:
parentdir

'/home/keisuke/Documents/Research/sshfs/DimerMaster'

In [16]:
fig = go.Figure()
y = np.log(np.abs(dimer_corr[0,0]))

fig.add_trace(
    go.Scatter(x = t_list, y = y)
)
fig.show()

NameError: name 'go' is not defined

In [71]:
t_list

array([0.        , 0.02040816, 0.04081633, 0.06122449, 0.08163265,
       0.10204082, 0.12244898, 0.14285714, 0.16326531, 0.18367347,
       0.20408163, 0.2244898 , 0.24489796, 0.26530612, 0.28571429,
       0.30612245, 0.32653061, 0.34693878, 0.36734694, 0.3877551 ,
       0.40816327, 0.42857143, 0.44897959, 0.46938776, 0.48979592,
       0.51020408, 0.53061224, 0.55102041, 0.57142857, 0.59183673,
       0.6122449 , 0.63265306, 0.65306122, 0.67346939, 0.69387755,
       0.71428571, 0.73469388, 0.75510204, 0.7755102 , 0.79591837,
       0.81632653, 0.83673469, 0.85714286, 0.87755102, 0.89795918,
       0.91836735, 0.93877551, 0.95918367, 0.97959184, 1.        ])

In [72]:
np.logical_and(0<t_list, t_list<1)

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False])

In [52]:
a = np.arange(16).reshape(4,4)

In [54]:
fft.fft2(a)

array([[120. -0.j,  -8. +8.j,  -8. -0.j,  -8. -8.j],
       [-32.+32.j,   0. +0.j,   0. +0.j,   0. -0.j],
       [-32. -0.j,   0. +0.j,   0. -0.j,   0. -0.j],
       [-32.-32.j,   0. +0.j,   0. -0.j,   0. -0.j]])

In [65]:
(a * np.array([1,1j,-1,-1j])).sum()

(-8-8j)

In [73]:
n = 10
L = 16
state = np.zeros(L, dtype=np.int8)
for i in range(L):
    state[i] = n // 2**(L-i)
    n %= 2**(L-i)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1], dtype=int8)

In [75]:
dft_matrix = np.fft.fft(np.eye(10))

In [77]:
dft_matrix[0]

array([1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
       1.+0.j, 1.+0.j])